In [1]:
import opensees
import opensees.analysis
import opensees.pattern
import quakeio.elcentro

In [2]:
material = opensees.uniaxial.Elastic(1, 3000.0)
truss = opensees.element.Truss(1, [1, 2], 10.0, material)

m = opensees.model(ndm=1, ndf=1,
    nodes = {1: [0.0], 2: [72.0]},
    elems = [[truss, [1,2]]]
)

m.fix(1,1)

In [3]:
patterns = {
    1: opensees.pattern.UniformExcitation(1, 1, accel=quakeio.elcentro.accel),
}

In [4]:
strategy = dict(
    constraints = ["Transformation"],
    numberer    = ["ParallelPlain"],
    test        = ["NormDispIncr", 1e-6, 6],
    system      = ['ProfileSPD'],
    algorithm   = ["Linear"],
    # integrator  = ["Newmark", 0.5, 0.25],
    # analysis    = ["Variable"]
)

In [5]:
a = opensees.analysis.DirectIntegrationAnalysis(m, strategy, patterns)

In [6]:
incr = 0.0001
steps = 5
a.analyze(steps, incr)

In [7]:
time = a.rt.time()
print(f'time: ', time)
approx_vtime = incr * steps
assert 0.99 < time / approx_vtime < 1.01,  (time,  approx_vtime)

time:  0.0005


## Part II


In [8]:
m.mass(2, [1.0])

In [9]:
hist = opensees.pattern.TimeSeries(step=0.1, values=[
         0.0000, 5.0000, 8.6603,10.0000, 8.6603, 5.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
])

patterns = {
    1: opensees.pattern.Plain(1, hist, {(2,"x"): 1.0})
}

</details>

In [10]:
patterns[1].series.values

[0.0, 5.0, 8.6603, 10.0, 8.6603, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [11]:
# ops.analysis('Transient')
a = opensees.analysis.DirectIntegrationAnalysis(m, strategy, patterns)

In [12]:
for i in range(10):
    a.analyze(1, 0.1)
    print(a.rt.time(), a.rt.getNodeResponse(2, "displ"))

0.1 [0.00612245]
0.2 [0.02259945]
0.30000000000000004 [0.03253137]
0.4 [0.02177143]
0.5 [0.00615625]
0.6 [0.00082664]
0.7 [-6.7537103e-05]
0.7999999999999999 [-0.00082388]
0.8999999999999999 [0.00010116]
0.9999999999999999 [0.00081975]
